In [1]:
import cobra
import numpy as np
import pandas as pd

In [2]:
def set_minimal_media(model,ions=[],auto_ions=True,ignore =[]):
    ''' set media to minimal (M9) FBA (no carbon source) by default it provides a predeterimined set of ions (
    auto_ions =True), you can specify which metabolites not to change for example if you have calibrated the 
    vmaxes or want anaerobic conditions'''
    if auto_ions == True:
        ions = ions + ['ca2_e', 'cl_e','cobalt2_e','cu2_e','fe2_e','fe3_e','h_e','h2o_e','k_e','mg2_e','mn2_e',
                       'mobd_e','na1_e','nh4_e','ni2_e',
                            'o2_e','pi_e','so4_e','zn2_e',
                            'tungs_e','sel_e','slnt_e','cbl1_e']
    exch = [x for x in model.reactions  if len(x.metabolites) == 1 and next(iter(x.metabolites.values())) == -1 and next(iter(x.metabolites.keys())).compartment == 'e']
    #If compartments are not assigned to the model 
    if len(exch) == 0:
        exch = [x for x in model.reactions  if len(x.metabolites) == 1 and next(iter(x.metabolites.values())) == -1 and next(iter(x.metabolites.keys())).id[-2:] == '_e']

    # .. media
    for l in exch:
        if next(iter(l.metabolites.keys())).id in ions:
            model.reactions.get_by_id(l.id).lower_bound = -1000
        elif next(iter(l.metabolites.keys())).id not in ignore:
            model.reactions.get_by_id(l.id).lower_bound = -0
        else:
            pass

### Create Models

Ok let's simplify this. I'm going to load the models and create 2 versions

one on glucose only
one on citrate only

I'm  also not going to add any side reaction as the side reactions activity is already covered by OAADC.

h_c + oaa_c ⇌ co2_c + pyr_c

NB this covers the side reaction of  both maeaA and maeaB:

https://www.uniprot.org/uniprot/P26616
https://www.uniprot.org/uniprot/P76558

In [3]:
'''load the base model for REL606 and create 2 versions of the model
a glucose only one and a citrate only one'''
base_model = cobra.io.read_sbml_model('iECB_1328.xml.gz')
set_minimal_media(base_model)
glucose_only = base_model.copy()
citrate_only = base_model.copy()
both = base_model.copy()
glucose_only.reactions.EX_glc__D_e.lower_bound=-10.0
citrate_only.reactions.EX_cit_e.lower_bound = -10.
both.reactions.EX_glc__D_e.lower_bound=-10.0
both.reactions.EX_cit_e.lower_bound=-10.0

Academic license - for non-commercial use only
Read LP format model from file /tmp/tmppk8co6dw.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpkultztum.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmpgvr4aw21.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros


# We'll perform pFBA on all three models. Lets  then see what happens to the main fluxes of interests

In [4]:
glc = cobra.flux_analysis.pfba(glucose_only)
cit = cobra.flux_analysis.pfba(citrate_only)
bt = cobra.flux_analysis.pfba(both)

In [5]:
# First lets get the biomass flux
print('Growth')
print(glc.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print(cit.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print('MaeA')
#MaeA activity on citrate but not on glucose
print(glc.get_primal_by_id('ME1'))
print(cit.get_primal_by_id('ME1'))
print('MaeB')
#MaeB activity on neither
print(glc.get_primal_by_id('ME2'))
print(cit.get_primal_by_id('ME2'))
print('MDH')
# MDH activity on both
print(glc.get_primal_by_id('MDH'))
print(cit.get_primal_by_id('MDH'))


Growth
0.9824784386660051
0.7183799682334098
MaeA
0.0
4.354100850130253
MaeB
0.0
0.0
MDH
2.675387722200581
4.012592471901773


### Lets repeat this but this time knocking out MaeA main activity

In [6]:
glucose_only.reactions.ME1.upper_bound=0.0
citrate_only.reactions.ME1.upper_bound=0.0
glc2 = cobra.flux_analysis.pfba(glucose_only)
cit2 = cobra.flux_analysis.pfba(citrate_only)

In [7]:
# biomas flux is the same
print('Growth')
print(glc2.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print(cit2.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print('MaeA')
#obviously no more meaA activity as we knocked it out
print(glc2.get_primal_by_id('ME1'))
print(cit2.get_primal_by_id('ME1'))
print('MaeB')
#MaeB activity replaces meaA 
print(glc2.get_primal_by_id('ME2'))
print(cit2.get_primal_by_id('ME2'))
print('MDH')
# MDH activity on both
print(glc2.get_primal_by_id('MDH'))
print(cit2.get_primal_by_id('MDH'))


Growth
0.9824784386660044
0.7183799682334069
MaeA
0.0
0.0
MaeB
0.0
4.354100850130364
MDH
2.675387722200487
4.012752670635938


### What happen if we also knock out ME2

In [8]:
glucose_only.reactions.ME2.upper_bound=0.0
citrate_only.reactions.ME2.upper_bound=0.0
glc3 = cobra.flux_analysis.pfba(glucose_only)
cit3 = cobra.flux_analysis.pfba(citrate_only)

In [ ]:
# First lets get the biomass flux
print('Growth')
print(glc3.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print(cit3.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print('MaeA')
##obviously no more meaA activity as we already knocked it out
print(glc3.get_primal_by_id('ME1'))
print(cit3.get_primal_by_id('ME1'))
print('MaeB')
#similarly no more meaB activity as we knocked it out
print(glc3.get_primal_by_id('ME2'))
print(cit3.get_primal_by_id('ME2'))
print('MDH')
# MDH activity on citrate goes up to replace MaeA
print(glc3.get_primal_by_id('MDH'))
print(cit3.get_primal_by_id('MDH'))
# The extra oxaloacetate is then converted to pyruvate 
# NB  MDH + OAADC is stoichiometrically equivalent to ME1. Both differes from ME2 which is NADP dependent rather than NAD 
print(glc3.get_primal_by_id('OAADC'))
print(cit3.get_primal_by_id('OAADC'))

### What happen if we force MDH activity to it's original level. i.e you only allow as much MDH activity  as is optimal when ME1 or ME2 are active. 

In [9]:
citrate_only.reactions.MDH.upper_bound = 4.012752670635938
glucose_only.reactions.MDH.upper_bound = 4.012752670635938
glc4 = cobra.flux_analysis.pfba(glucose_only)
cit4 = cobra.flux_analysis.pfba(citrate_only)

In [10]:
# Now we see a weak growth cost(it's weak because there are still other slightly less optimal ways to use the exces malate (see next chunk)
print('Growth')
print(glc4.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print(cit4.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print('MaeA')
#MaeA activity on citrate but not on glucose
print(glc4.get_primal_by_id('ME1'))
print(cit4.get_primal_by_id('ME1'))
print('MaeB')
#MaeB activity on either
print(glc4.get_primal_by_id('ME2'))
print(cit4.get_primal_by_id('ME2'))
print('MDH')
# MDH activity on both
print(glc4.get_primal_by_id('MDH'))
print(cit4.get_primal_by_id('MDH'))

Growth
0.9824784386660055
0.7039804057371246
MaeA
0.0
0.0
MaeB
0.0
0.0
MDH
2.675606814890528
4.012752670635938


### Here are a bunch of other malate consuming reactions that increase in flux and thus buffer the loss of ME1 (targets for alternative pathways)

In [11]:
glucose_only.reactions.MDH2.upper_bound=0.0
glucose_only.reactions.MDH3.upper_bound=0.0
glucose_only.reactions.DMALRED.upper_bound=1.5702995887617277
glucose_only.reactions.MOX.upper_bound = 0.001168085828346487
citrate_only.reactions.MDH3.upper_bound=0.0
citrate_only.reactions.DMALRED.upper_bound=1.5702995887617277
citrate_only.reactions.MOX.upper_bound = 0.001168085828346487
glc4 = cobra.flux_analysis.pfba(glucose_only)
cit4 = cobra.flux_analysis.pfba(citrate_only)

In [12]:
# You can see the drop in growth now if you knock them or force them to their base level (i.e level one ME1 is active)
print('Growth')
print(glc4.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))
print(cit4.get_primal_by_id('BIOMASS_Ec_iJO1366_core_53p95M'))

Growth
0.9824782205471678
0.7039804057371277


### Finally  to really prove that the first three ways of handling malate are truuly equivalent lets quickly do FVA

In [13]:
#Reset everything
glucose_only = base_model.copy()
citrate_only = base_model.copy()
glucose_only.reactions.EX_glc__D_e.lower_bound = -10.0
citrate_only.reactions.EX_cit_e.lower_bound=-10.0

Read LP format model from file /tmp/tmpoc6r3akj.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros
Read LP format model from file /tmp/tmp8msihqa_.lp
Reading time = 0.01 seconds
: 1951 rows, 5496 columns, 21258 nonzeros


In [14]:
#We never expect any activity on Glucose
ME_reactions = [glucose_only.reactions.ME1, glucose_only.reactions.ME2,glucose_only.reactions.MDH]
cobra.flux_analysis.flux_variability_analysis(glucose_only, reaction_list=ME_reactions, loopless=False)

,minimum,maximum
ME1,0.000000,0.000000e+00
ME2,0.000000,1.089795e-12
MDH,2.675388,4.823197e+00


In [15]:
# But we can expect activity on any of these reactions in glucose
cobra.flux_analysis.flux_variability_analysis(citrate_only, reaction_list=ME_reactions, loopless=False)

,minimum,maximum
ME1,0.000000,4.354101
ME2,0.000000,4.354101
MDH,4.012592,10.336974
